# Treinando Movie Plots from Wikipedia

## Bibliotecas

In [1]:
import pandas as pd

import nltk
from nltk.corpus import stopwords

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
debugging = True
# debugging = False

## Base de dados

In [4]:
df = pd.read_csv("Dados/wiki_movie_plots_deduped.csv")
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [5]:
corpus = df["Plot"]
corpus.head()

0    A bartender is working at a saloon, serving dr...
1    The moon, painted with a smiling face hangs ov...
2    The film, just over a minute long, is composed...
3    Lasting just 61 seconds and consisting of two ...
4    The earliest known adaptation of the classic f...
Name: Plot, dtype: object

### Remoção de stopwords e tokenização

In [6]:
stopwords = stopwords.words('english')
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [7]:
corpus = list(corpus)

if debugging:
    plots = [nltk.word_tokenize(word.lower()) for word in corpus[:5] if not word.lower() in stopwords]
else:
    plots = [nltk.word_tokenize(word.lower()) for word in corpus if not word.lower() in stopwords]

plots[0]

['a',
 'bartender',
 'is',
 'working',
 'at',
 'a',
 'saloon',
 ',',
 'serving',
 'drinks',
 'to',
 'customers',
 '.',
 'after',
 'he',
 'fills',
 'a',
 'stereotypically',
 'irish',
 'man',
 "'s",
 'bucket',
 'with',
 'beer',
 ',',
 'carrie',
 'nation',
 'and',
 'her',
 'followers',
 'burst',
 'inside',
 '.',
 'they',
 'assault',
 'the',
 'irish',
 'man',
 ',',
 'pulling',
 'his',
 'hat',
 'over',
 'his',
 'eyes',
 'and',
 'then',
 'dumping',
 'the',
 'beer',
 'over',
 'his',
 'head',
 '.',
 'the',
 'group',
 'then',
 'begin',
 'wrecking',
 'the',
 'bar',
 ',',
 'smashing',
 'the',
 'fixtures',
 ',',
 'mirrors',
 ',',
 'and',
 'breaking',
 'the',
 'cash',
 'register',
 '.',
 'the',
 'bartender',
 'then',
 'sprays',
 'seltzer',
 'water',
 'in',
 'nation',
 "'s",
 'face',
 'before',
 'a',
 'group',
 'of',
 'policemen',
 'appear',
 'and',
 'order',
 'everybody',
 'to',
 'leave',
 '.',
 '[',
 '1',
 ']']

### Tamanho de treino e teste

In [8]:
size = int(len(plots) * 0.2)
treino = plots[size:]
teste = plots[:size]

len(treino), len(teste)

(4, 1)

## CBOW - Continuos Bag-of-words
Rede neural proposta para o treinamento de word embeddings livre de contexto. O modelo CBOW aprende representações vetoriais ao ser treinado na tarefa de prever um determinado token-alvo dado os tokens ao seu redor.

Vamos começar definindo o vocabulário para o qual treinaremos representações vetoriais:

In [9]:
def get_vocab(texts):

    vocab = []

    for row in texts:
        vocab.extend(row)

    vocab = list(set(vocab))
    vocab += ['<pad>', '<oov>'] # padding & out of vocabulary

    w2id = { w:i for i, w in enumerate(vocab) } # word -> id
    id2w = { i:w for i, w in enumerate(vocab) } # id -> word

    return vocab, w2id, id2w

In [10]:
vocab, w2id, id2w = get_vocab(treino)
print('Número de palavras: ', len(vocab))

Número de palavras:  247


Vamos definir então os exemplos de treinamento. Como mencionado acima, as redes neurais CBOW aprendem os word embeddings ao serem treinadas na tarefa de prever um token dado um contexto de tokens ao redor. Portanto, devemos construir instâncias compostas de um token e seus tokens vizinhos de acordo com um tamanho de janela ( window size ). Neste caso, vamos definir uma janela de tamanho 2, isto é, vamos construir um contexto compostos pelos 2 tokens anteriores e os 2 tokens posteriores de um token alvo:

In [11]:
def context_window(tokens, size = 3):

    tokens = (['<pad>'] * size) + tokens + (['<pad>'] * size)

    contexts = []

    for i in range(size, len(tokens) - size):
        context = tokens[i-size : i] + tokens[i+1 : i+size+1]
        word = tokens[i]
        contexts.append({ 'context': ' '.join(context), 'word': word })

    return contexts

In [12]:
data = []
window_size = 3

for plot in treino:
    data.extend(context_window(plot, size = window_size))

In [13]:
data[10]

{'context': 'face hangs over park at night', 'word': 'a'}

In [14]:
data[100]

{'context': '<pad> <pad> the , just over', 'word': 'film'}

Vamos definir o modelo utilizando Pytorch:

In [15]:
class CBOW(nn.Module):

    def __init__(self, inp_dim, nvocab, window_size, w2id, device):
        '''
        Inicializando uma rede neural CBOW

        Parâmetros:
        - inp_dim: dimensão dos embeddings
        - nvocab: tamanho do vocabulário de palavras para as quais treinaremos word embeddings
        - window_size: tamanho da janela de contexto
        - w2id: mapping de um token para seu índice na matriz de embeddings
        - device: dispositivo onde a rede neural será alocada (cpu ou cuda)
        '''
        super(CBOW, self).__init__()

        self.device = device
        self.w2id = w2id
        self.lookup = nn.Embedding(nvocab, inp_dim)
        self.weight = nn.Linear(2 * window_size * inp_dim, nvocab)
        self.softmax = nn.LogSoftmax(1)

    def forward(self, X):
        '''
        Forward pass

        Parâmetros:
        - X: lista de contextos de entrada

        Return:
        Probabilidade entre as palavras do vocabulário
        '''
        contexts = []

        for context in X:
            indexes = []

            for token in context.split():
                try:
                    indexes.append(w2id[token])
                except:
                    indexes.append(w2id['<oov>']) # out of vocabulary
            
            contexts.append(indexes)

        contexts = torch.tensor(contexts).to(self.device)
        embeddings = self.lookup(contexts)

        batch_size, window_size, inp_dim = embeddings.size()
        concatenation = embeddings.view(batch_size, window_size * inp_dim)

        z = self.weight(concatenation)

        return self.softmax(z)

Definindo os parâmetros da rede neural e de treinamento

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inp_dim = 300
nvocab = len(vocab)
nepochs = 1

if debugging:
    batch_size = 1 
    batch_status = 1
else:
    batch_size = 256 
    batch_status = 256
    
learning_rate = 0.01
window_size = window_size

Inicializando o modelo, a função de erro e o otimizador

In [17]:
model = CBOW(
    inp_dim,
    nvocab,
    window_size = window_size,
    w2id = w2id,
    device = device
).to(device)

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.parameters(), lr = learning_rate)

Separando o conjunto de treinamento em lotes ( batches )

In [18]:
train_data = DataLoader(data, batch_size = batch_size, shuffle = True)

Treinando

In [19]:
for epoch in range(nepochs):

    losses = []

    for batch_index, row in enumerate(train_data):
        X = row['context']
        y = []

        for word in row['word']:
            try:
                y.append(w2id[word])
            except:
                y.append(w2id['oov'])

        y = torch.tensor(y).to(device)

        # Forward
        outputs = model(X)

        # Loss
        loss = criterion(outputs, y)
        losses.append(float(loss))

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Display
        if(batch_index + 1) % batch_status == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTotal Loss: {:.6f}'.format(
                epoch + 1, batch_index + 1, len(train_data),
                100. * batch_index / len(train_data), float(loss), 
                round(sum(losses) / len(losses), 5))
            )

Train Epoch: 1 [1/526 (0%)]	Loss: 4.783633	Total Loss: 4.783630
Train Epoch: 1 [2/526 (0%)]	Loss: 5.275222	Total Loss: 5.029430
Train Epoch: 1 [3/526 (0%)]	Loss: 5.037700	Total Loss: 5.032190
Train Epoch: 1 [4/526 (1%)]	Loss: 5.634353	Total Loss: 5.182730
Train Epoch: 1 [5/526 (1%)]	Loss: 5.582797	Total Loss: 5.262740
Train Epoch: 1 [6/526 (1%)]	Loss: 5.986785	Total Loss: 5.383410
Train Epoch: 1 [7/526 (1%)]	Loss: 6.219871	Total Loss: 5.502910
Train Epoch: 1 [8/526 (1%)]	Loss: 5.099916	Total Loss: 5.452530
Train Epoch: 1 [9/526 (2%)]	Loss: 7.591757	Total Loss: 5.690230
Train Epoch: 1 [10/526 (2%)]	Loss: 22.767555	Total Loss: 7.397960
Train Epoch: 1 [11/526 (2%)]	Loss: 6.494330	Total Loss: 7.315810
Train Epoch: 1 [12/526 (2%)]	Loss: 5.987987	Total Loss: 7.205160
Train Epoch: 1 [13/526 (2%)]	Loss: 15.628428	Total Loss: 7.853100
Train Epoch: 1 [14/526 (2%)]	Loss: 5.464654	Total Loss: 7.682500
Train Epoch: 1 [15/526 (3%)]	Loss: 5.426507	Total Loss: 7.532100
Train Epoch: 1 [16/526 (3%)]	Los

In [20]:
embeddings = model.lookup.weight.data.cpu().numpy()

w2emb = { w: emb for (w, emb) in zip(vocab, list(embeddings)) }

In [21]:
lookup_word = 'assassin'
similarities = cosine_similarity([w2emb[lookup_word]], embeddings)[0]

candidates = sorted([(vocab[i], sim) for i, sim in enumerate(similarities)], 
    key = lambda x: x[1], reverse = True)[:10]

for candidate in candidates:
    print(candidate)

('assassin', 1.0)
('falls', 0.1336894)
('they', 0.13009809)
('girl', 0.1295644)
('with', 0.12798722)
('61', 0.122388855)
('escapes', 0.11681093)
('films', 0.11048595)
('which', 0.110460654)
('fans', 0.11045283)


## Skip-gram
Arquitetura proposta junto com a CBOW. Contudo, a skip-gram aprende representações vetoriais ao ser treinada prevendo os tokens ao redor de um token alvo.

Como já definimos o vocabulário anteriormente, vamos criar os exemplos para o treinamento da arquitetura. Note que agora daremos o token alvo como entrada e orientaremos a rede neural a prever as palavras ao redor:

In [22]:
def context_window(tokens, size = 3):

    tokens = (['<pad>'] * size) + tokens + (['<pad>'] * size)

    contexts = []

    for i in range(size, len(tokens) - size):
        context = tokens[i-size : i] + tokens[i+1 : i+size+1]
        word = tokens[i]
        contexts.extend([{ 'word': word, 'context': ctoken } for ctoken in context])

    return contexts

In [23]:
data = []
window_size = 10

for plot in treino:
    data.extend(context_window(plot, size = window_size))

In [24]:
data[8:20]

[{'word': 'the', 'context': '<pad>'},
 {'word': 'the', 'context': '<pad>'},
 {'word': 'the', 'context': 'moon'},
 {'word': 'the', 'context': ','},
 {'word': 'the', 'context': 'painted'},
 {'word': 'the', 'context': 'with'},
 {'word': 'the', 'context': 'a'},
 {'word': 'the', 'context': 'smiling'},
 {'word': 'the', 'context': 'face'},
 {'word': 'the', 'context': 'hangs'},
 {'word': 'the', 'context': 'over'},
 {'word': 'the', 'context': 'a'}]

Definindo o modelo

In [25]:
class SkipGram(nn.Module):

    def __init__(self, inp_dim, nvocab, w2id, device):
        '''
        Inicializando a arquitetura skip-gram

        Parâmetros:
        - inp-dim: dimensão dos word embeddings
        - nvocab: tamanho do vocabulário de palavras
        - w2id: mapping de um token para seu índice na matriz de embeddings
        - device: hardware onde o modelo será alocado (cpu ou cuda)
        '''
        super(SkipGram, self).__init__()

        self.device = device
        self.w2id = w2id
        self.lookup = nn.Embedding(nvocab, inp_dim)
        self.weight = nn.Linear(inp_dim, nvocab)
        self.softmax = nn.LogSoftmax(1)

    def forward(self, X):
        '''
        Forward pass

        Parâmetros:
        - X: tokens-alvo de entrada
        '''
        words = []

        for token in X:
            try:
                words.append(w2id[token])
            except:
                words.append(w2id['<oov>'])

        words = torch.tensor(words).to(self.device)
        embeddings = self.lookup(words)

        z = self.weight(embeddings)

        return self.softmax(z)

Parâmetros da rede neural

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inp_dim = 300
nvocab = len(vocab)
nepochs = 1

if debugging:
    batch_size = 1 
    batch_status = 1
else:
    batch_size = 256
    batch_status = 512
 
learning_rate = 0.01
window_size = window_size

Inicializando o modelo, a função de erro e o otimizador

In [27]:
model = SkipGram(
    inp_dim,
    nvocab,
    w2id = w2id,
    device = device
).to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0)

Separando os dados de treino em lotes ( batches )

In [28]:
train_data = DataLoader(data, batch_size = batch_size, shuffle = True)

Treinando o modelo

In [29]:
for epoch in range(nepochs):

    losses = []

    for batch_index, row in enumerate(train_data):
        X = row['word']
        y = []

        for context in row['context']:
            try:
                y.append(w2id[context])
            except:
                y.append(w2id['oov'])
    
        y = torch.tensor(y).to(device)

        # Forward
        outputs = model(X)
        
        # Calculate loss
        loss = criterion(outputs, y)
        losses.append(float(loss))
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Display
        if (batch_index + 1) % batch_status == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTotal Loss: {:.6f}'.format(
                epoch + 1, batch_index + 1, len(train_data),
                100. * batch_index / len(train_data), float(loss), 
                round(sum(losses) / len(losses), 5))
            )

Train Epoch: 1 [1/10520 (0%)]	Loss: 5.228788	Total Loss: 5.228790
Train Epoch: 1 [2/10520 (0%)]	Loss: 5.595629	Total Loss: 5.412210
Train Epoch: 1 [3/10520 (0%)]	Loss: 4.297654	Total Loss: 5.040690
Train Epoch: 1 [4/10520 (0%)]	Loss: 5.782529	Total Loss: 5.226150
Train Epoch: 1 [5/10520 (0%)]	Loss: 4.989084	Total Loss: 5.178740
Train Epoch: 1 [6/10520 (0%)]	Loss: 8.788565	Total Loss: 5.780370
Train Epoch: 1 [7/10520 (0%)]	Loss: 13.659849	Total Loss: 6.906010
Train Epoch: 1 [8/10520 (0%)]	Loss: 5.298081	Total Loss: 6.705020
Train Epoch: 1 [9/10520 (0%)]	Loss: 4.792071	Total Loss: 6.492470
Train Epoch: 1 [10/10520 (0%)]	Loss: 5.785362	Total Loss: 6.421760
Train Epoch: 1 [11/10520 (0%)]	Loss: 6.299594	Total Loss: 6.410660
Train Epoch: 1 [12/10520 (0%)]	Loss: 3.667639	Total Loss: 6.182070
Train Epoch: 1 [13/10520 (0%)]	Loss: 7.009599	Total Loss: 6.245730
Train Epoch: 1 [14/10520 (0%)]	Loss: 5.503517	Total Loss: 6.192710
Train Epoch: 1 [15/10520 (0%)]	Loss: 11.633988	Total Loss: 6.555460
Tr

In [30]:
embeddings = model.lookup.weight.data.cpu().numpy()

w2emb = { w:emb for (w, emb) in zip(vocab, list(embeddings)) }

In [31]:
lookup_word = 'assassin'

similarities = cosine_similarity([w2emb[lookup_word]], embeddings)[0]

candidates = sorted([(vocab[i], sim) for i, sim in enumerate(similarities)], 
    key=lambda x: x[1], reverse=True)[:10]

for candidate in candidates:
  print(candidate)

('assassin', 0.9999998)
('over', 0.43968943)
('runs', 0.3026252)
('adaptation', 0.2911501)
('signs', 0.27399758)
('seconds', 0.2446576)
('may', 0.21178326)
('smiles', 0.19291343)
('shot', 0.19270724)
('minute', 0.16397943)
